- Author: Umair Khan
- Data: 31-07-2020

# 1. Imports

In [1]:
import sys
# Adding root to system path
sys.path.append('..')
from models.conditional_gan import CGanGenerator, CGanDiscriminator, CGan, GanModel
from data_wrangling.data_loader import DataLoader
from util import TrainParam, trainer
import json
from numpy.random import randn, randint
from scipy.interpolate import interp1d
import numpy as np
from matplotlib import pyplot


Using TensorFlow backend.


# 2. Load Data & Models

In [2]:
config_path = '..\\config\\cgan_config.json'
with open(config_path) as config_file:
    config = json.load(config_file)

train_param = TrainParam(
        n_epochs=2000,
        batch_size=128,
        latent_dim=100,
        epoch_checkpoint=1,
        n_summary_samples=36,
        config=config,
    )

data_loader = DataLoader(data_path=config['data_path'])
discriminator = CGanDiscriminator(
        input_shape=(96,96,3),
        n_classes=2
    )()
generator = CGanGenerator(latent_dimension=100, n_classes=2)()

#discriminator.load_weights('..\\training_output\\archive\\model_working_epochs_500\\m_model_399.h5')
#generator.load_weights('..\\training_output\\archive\\model_working_epochs_500\\g_model_499.h5')

gan = CGan(generator=generator, discriminator=discriminator)()
gan_model = GanModel(
        generator=generator,
        discriminator=discriminator,
        gan=gan
    )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# 3. Train

In [ ]:
epoch_dloss_r, epoch_dloss_f, epoch_gen_loss = trainer(
    gan_model=gan_model, data_loader=data_loader, tarin_param=train_param)

Instructions for updating:
Use tf.cast instead.
>Epoch:1 DR[1.738], DF[1.787] GL[2.336]]
>Epoch:2 DR[1.034], DF[0.923] GL[1.385]]
>Epoch:3 DR[0.950], DF[0.238] GL[0.429]]
>Epoch:4 DR[0.856], DF[0.126] GL[2.058]]
>Epoch:5 DR[0.806], DF[0.120] GL[2.974]]
>Epoch:6 DR[0.792], DF[0.123] GL[4.947]]]
>Epoch:7 DR[0.997], DF[0.385] GL[1.768]]]
>Epoch:8 DR[0.738], DF[0.008] GL[0.199]]


In [3]:
discriminator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 48, 48, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 48, 48, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 48, 48, 64)   0           leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
conv2d_2 (

c:\users\320048294\appdata\local\programs\python\python36\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


# 3. Visualization

In [ ]:
generator.load_weights('..\\training_output\\model\\g_model_521.h5')
vectors = randn(21*100).reshape(21, 100)
linfit_vectors = [interp1d([1,10], vectors[i:i+2, :], axis=0) for i in range(0, 20)]
interpolated_vectors = np.vstack([linfit_vectors[j]([i+1 for i in range(0, 10)]) for j in range(len(linfit_vectors))])
X = generator.predict([interpolated_vectors, np.repeat(randint(0, 2, 20), 10)])
X = ((X * 127.5) + 127.5).astype('uint8')
pyplot.figure(figsize=(25, 100))
for i in range(200):
    # define subplot
    pyplot.subplot(len(linfit_vectors), 10, 1 + i)
    # turn off axis
    pyplot.axis('off')
    # plot raw pixel data
    pyplot.imshow(X[i, :, :, :])